# Risk-Neutral Valuation and Numerical Methods

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Load the Libs we need

In [1]:
# Importing Libraries
import numpy as np

# Importing specific modules and functions
from scipy.linalg import solve

### Risk-Neutral Valuation Principles for Options Pricing

#### Simulating Price Paths

In [2]:
def mb_simulate_price_paths(S0, r, sigma, T, N, M, seed=None):
    if seed is not None:
        np.random.seed(seed)  # Set the seed for reproducibility
    dt = T / N
    price_paths = np.zeros((N + 1, M))
    price_paths[0] = S0
    for t in range(1, N + 1):
        z = np.random.standard_normal(M)
        price_paths[t] = price_paths[t-1] * np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * z)
    return price_paths

#### Pricing a Binary Option

In [3]:
def mb_price_binary_option(S0, K, r, sigma, T, N, M, option_type='call', seed=None):
    # Simulate price paths with the given seed
    price_paths = mb_simulate_price_paths(S0, r, sigma, T, N, M, seed=seed)
    
    # Determine the payoff based on the option type
    if option_type == 'call':
        payoff = np.where(price_paths[-1] > K, 1, 0)  # Payoff is 1 if final price > strike for call
    elif option_type == 'put':
        payoff = np.where(price_paths[-1] < K, 1, 0)  # Payoff is 1 if final price < strike for put
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    # Calculate the binary option price
    binary_price = np.exp(-r * T) * np.mean(payoff)  # Discounted average payoff
    return binary_price

# Example usage with a fixed seed
S0 = 100     # Initial stock price
K = 105      # Strike price
r = 0.05     # Risk-free rate
sigma = 0.2  # Volatility
T = 1        # Time to maturity in years
N = 252      # Number of time steps
M = 10000    # Number of simulation paths
seed = 42    # Random seed for reproducibility

binary_call_option_price = mb_price_binary_option(S0, K, r, sigma, T, N, M, option_type='call', seed=seed)
binary_put_option_price = mb_price_binary_option(S0, K, r, sigma, T, N, M, option_type='put', seed=seed)

print(f"The price of the binary call option is: ${binary_call_option_price:.4f}")
print(f"The price of the binary put option is: ${binary_put_option_price:.4f}")


The price of the binary call option is: $0.4371
The price of the binary put option is: $0.5141


#### Pricing a Barrier Option

In [4]:
def mb_price_barrier_option(S0, K, B, r, sigma, T, N, M, option_type='call', seed=None):
    # Simulate price paths with the given seed
    price_paths = mb_simulate_price_paths(S0, r, sigma, T, N, M, seed=seed)
    
    # Determine the payoff based on the option type and barrier condition
    if option_type == 'call':
        # Payoff for a call: price is greater than strike and below the barrier
        payoff = np.where((price_paths[-1] > K) & (np.all(price_paths <= B, axis=0)), price_paths[-1] - K, 0)
    elif option_type == 'put':
        # Payoff for a put: price is less than strike and above the barrier
        payoff = np.where((price_paths[-1] < K) & (np.all(price_paths >= B, axis=0)), K - price_paths[-1], 0)
    else:
        raise ValueError("option_type must be 'call' or 'put'")
    
    # Calculate the barrier option price
    barrier_price = np.exp(-r * T) * np.mean(payoff)  # Discounted average payoff
    return barrier_price

# Example usage with a fixed seed
S0 = 100     # Initial stock price
K = 105      # Strike price
B = 110      # Barrier level
r = 0.05     # Risk-free rate
sigma = 0.2  # Volatility
T = 1        # Time to maturity in years
N = 252      # Number of time steps
M = 10000    # Number of simulation paths
seed = 42    # Random seed for reproducibility

barrier_call_option_price = mb_price_barrier_option(S0, K, B, r, sigma, T, N, M, option_type='call', seed=seed)
barrier_put_option_price = mb_price_barrier_option(S0, K, B, r, sigma, T, N, M, option_type='put', seed=seed)

print(f"The price of the barrier call option is: ${barrier_call_option_price:.4f}")
print(f"The price of the barrier put option is: ${barrier_put_option_price:.4f}")


The price of the barrier call option is: $0.0253
The price of the barrier put option is: $0.0000


## Implementing numerical methods for options pricing in Python

#### Constructing the Crank-Nicolson Scheme

In [5]:
# Parameters
S_max = 300
K = 145
r = 0.05
sigma = 0.2
T = 1
N_S = 100
N_T = 1000
dt = T / N_T
ds = S_max / N_S

# Set up A and B matrices
A = np.zeros((N_S - 2, N_S - 2))
B = np.zeros((N_S - 2, N_S - 2))

for i in range(1, N_S - 1):
    j = i - 1
    A[j, j] = 1 + dt * (sigma**2 * i**2 + r)
    B[j, j] = 1 - dt * (sigma**2 * i**2 + r)
    if i > 1:
        A[j, j - 1] = -0.5 * dt * (sigma**2 * i**2 - r * i)
        B[j, j - 1] = 0.5 * dt * (sigma**2 * i**2 - r * i)
    if i < N_S - 2:
        A[j, j + 1] = -0.5 * dt * (sigma**2 * i**2 + r * i)
        B[j, j + 1] = 0.5 * dt * (sigma**2 * i**2 + r * i)

# Initial condition: payoff at maturity for a call option
V_call = np.maximum(np.arange(0, S_max, ds) - K, 0)[1:-1]

# Initial condition: payoff at maturity for a put option
V_put = np.maximum(K - np.arange(0, S_max, ds), 0)[1:-1]

# Time-stepping loop for the call option
for t in range(N_T):
    b_call = B @ V_call
    # Adjust for boundary conditions if necessary
    V_call = solve(A, b_call)

# Time-stepping loop for the put option
for t in range(N_T):
    b_put = B @ V_put
    # Adjust for boundary conditions if necessary
    V_put = solve(A, b_put)

# Option prices at S = $150
call_option_price = V_call[int(150 / ds) - 1]
put_option_price = V_put[int(150 / ds) - 1]

print(f"The European call option price is: ${call_option_price:.2f}")
print(f"The European put option price is: ${put_option_price:.2f}")

The European call option price is: $23.24
The European put option price is: $8.14
